In [ ]:
import urllib.request
import re
import pandas as pd
import json
import numpy as np
import pickle
from nltk import tokenize
from nltk.tokenize import TreebankWordTokenizer
from nltk.tag import pos_tag

In [ ]:
with open('test_contract.json', encoding='utf-8') as f:
    lines=json.load(f)
lines = lines['elements']
print(lines)

In [ ]:
def tokenize_pos(data):
    context_pos = []
    sentences = tokenize.sent_tokenize(data)

    text_tokens = []
    for sent in sentences:
        text_tokens.append(TreebankWordTokenizer().tokenize(sent))

    tagged_sentences = []
    for sentencetokens in text_tokens:
        tagged_sentences.append(pos_tag(sentencetokens))
    context_pos.append(tagged_sentences)
    return context_pos

In [ ]:
def enumeration_identifier(data):
    text = data
    flag = 0
    # for i in range(len(text) - 4):
        #sub headings
    if text[0].isdigit() and text[1] == "." and text[2].isdigit() and text[2] != "0":
        flag = 2
    elif text[0].isdigit() and text[1] == "." and text[2].isdigit() and text[3] == " ":
        flag = 2
    elif text[0].isdigit() and text[1] == "." and text[2].isdigit() and text[3] == ".":
        flag = 2

    if text[0].isdigit() and text[1] == "." and text[2].isalpha():
        flag = 1
    elif text[0].isdigit() and text[1].isdigit() and text[2] == "." and text[3].isalpha():
        flag = 1

    # 1 digit enumeration
    if text[0].isdigit() and text[1] == "." and text[2] == " ":
        flag = 1
    # 2 digit enumeration
    elif text[0].isdigit() and text[1].isdigit() and text[2] == "." and text[3] == " ":
        flag = 1

    # 1 digit enumeration fpr 1.0 type heading
    if text[0].isdigit() and text[1] == "." and text[2] == "0":
        flag = 1
    # 2 digit enumeration for 10.0 type heading
    elif text[0].isdigit() and text[1].isdigit() and text[2] == "." and text[3] == "0":
        flag = 1

    # enumeration with parentheses
    if text[0].isdigit() and text[1] == ")":
        flag = 1
    elif text[0].isdigit() and text[1].isdigit() and text[2] == ")":
        flag = 1
        
    if text[0].isdigit() and text[1]==" ":
        flag=1
    elif text[0].isdigit() and text[1].isdigit() and text[2]==" " and text[3].isalpha():
        flag=1
    
    # 1 roman enumeration
    if isroman(text[0]) and text[1] == "." and text[2] == " ":
        flag = 1
    # 2 roman enumeration
    elif isroman(text[0]) and isroman(text[1]) and text[2] == "." and text[3] == " ":
        flag = 1
    # 3 roman enumeration
    elif isroman(text[0]) and isroman(text[1]) and isroman(text[2]) and text[3] == "." and text[4] == " ":
        flag = 1
    # 4 roman enumeration
    elif isroman(text[0]) and isroman(text[1]) and isroman(text[2]) and isroman(text[3]) and text[4] == "." and text[5] == " ":
        flag = 1

    if "ARTICLE" in text:
        flag = 1

    return flag

In [ ]:
def isroman(char):
    romans = ['I', 'V', 'X']
    if char in romans:
        return True
    else:
        return False

In [ ]:
def case_identifier(data):
    if data.isupper():
        case = 1
    elif data.islower():
        case = 2
    elif is_sent_title(data) == 0:
        case = 3
    elif is_sent_title(data) == 1:
        case = 4
    return case


In [ ]:
def is_sent_title(inp):
    sent = inp.split()
    flag = 0
    for word in sent:
        if word[0].isupper():
            flag += 1
        else:
            flag = 0
    try:
        return int(flag / len(sent))
    except:
        return 0

In [ ]:
def feature_vectors(context_pos, data):

    for context in context_pos:
        nouns = 0
        verbs = 0
        cardinal = 0
        words = 0
        punct = 0

        for con in context:
            for token in con:
                words += 1

                if 'NN' in token[1]:
                    nouns += 1

                if 'VB' in token[1]:
                    verbs += 1

                if 'CD' in token[1]:
                    cardinal += 1

                if '.' in token[1]:
                    punct += 1

    case = case_identifier(data)
    enumeration = enumeration_identifier(data)
    return words, nouns, verbs, cardinal, punct, enumeration, case

In [ ]:
#appending only required json lines
print(len(lines))
cleaned_lines = []


for line in lines:
    try:
        text = line['Text']
        cleaned_lines.append(line)
    except Exception as e:
        print(line)
print(len(cleaned_lines))

In [ ]:
text_info = []
for i in range(len(cleaned_lines)-1):
    line = cleaned_lines[i]
    flag = 0
    font = line['Font']
    next_font = cleaned_lines[i+1]['Font']
    current = cleaned_lines[i]['Text']
    next_line = cleaned_lines[i+1]['Text']
    
    if enumeration_identifier(current) == 1 and enumeration_identifier(next_line) != 1 and len(current) <6:
        enumeration = current
        size = int(line['TextSize'])
        if len(next_line.split(":"))>=1:
            heading = next_line.split(":")[0]
            body = " ".join(next_line.split(":")[1:])
        elif len(next_line.split("."))>1: 
            heading = next_line.split(".")[0]
            body = " ".join(next_line.split(".")[1:])
        #print(cleaned_lines[i+1]['Text'])
        if "Bold" in font['name'] or "F5" in font['name'] or "F3" in font['name'] or "F4" in font['name'] or "Bold" in next_font['name']:
            bold = 1
        else:
            bold = 0

        text_info.append([enumeration + heading, bold, size])
        if len(body)>4:
            text_info.append([body, bold, size])
     
        
    elif enumeration_identifier(current) == 1 and len(current) > 5:
        size = int(line['TextSize'])
        if len(current.split(":"))>=1:
            heading = current.split(":")[0]
            body = " ".join(current.split(":")[1:])
        elif len(cleaned_lines[i+1]['Text'].split(".")):
            heading = ".".join(current.split(".")[:2])
            body = " ".join(current.split(".")[2:])
            
        if "Bold" in font['name'] or "F5" in font['name'] or "F3" in font['name'] or "F4" in font['name'] or "Bold" in next_font['name']:
            bold = 1
        else:
            bold = 0
#         bold = 1
#         print(heading)
        text_info.append([heading, bold, size])
        if len(body)>4:
            text_info.append([body, bold, size])
    
    elif  len(current.split(" "))<5 and enumeration_identifier(current)!=1 and enumeration_identifier(next_line)==1:
        heading = next_line.split(".")[0] + ". "+ current
        if "Bold" in font['name'] or "F5" in font['name'] or "F3" in font['name'] or "F4" in font['name'] or "Bold" in next_font['name']:
            bold = 1
        else:
            bold = 0
        size = int(line['TextSize'])
        text_info.append([heading, bold, size])
    
    else: 
        if "Bold" in font['name'] or "F5" in font['name'] or "F3" in font['name']:
            bold = 1
        else:
            bold = 0
        size = int(line['TextSize'])
#         print(cleaned_lines[i]['Text'])
        if len(current)>4:
            text_info.append([current, bold, size])
        
    
print(text_info)



In [ ]:
cleaned_text_info = []
for text in text_info:
    if len(text[0])>6:
        cleaned_text_info.append(text)
    else:
        print(text[0])

In [ ]:
model_input = []
for info in cleaned_text_info:
    context_pos = tokenize_pos(info[0])
    words, nouns, verbs, cardinal, punct, enumeration, case = feature_vectors(context_pos, info[0])
    model_input.append([info[1], info[2], words, nouns, verbs, cardinal, punct, enumeration, case])
print(len(model_input))
print(len(text_info))

In [ ]:
filename = 'models/heading_det_model_gradboost1.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
results = []
for i in range(len(model_input)):
    val_data = np.array(model_input[i], dtype=float)
    pred = loaded_model.predict([val_data])
    text = cleaned_text_info[i][0]
    if pred:
        enum_flag=enumeration_identifier(text)
        if enum_flag==1:
            proc_pred = 1
        else:
            proc_pred = 0
    else:
        proc_pred = 0
    
    if proc_pred == 1: 
        print(val_data, proc_pred, text)
    results.append([proc_pred, text])

In [ ]:
for res in results:
    print(res)